In [172]:
import pandas as pd
import numpy as np
import geopandas 

In [67]:
nigeria_main_df = pd.read_stata('./Datasets/Nigeria_MTF_data/MTF_NG_HH_SEC_B.dta', convert_categoricals=False).drop(columns=['B_start']).drop(236)
nigeria_main_df.head()

nigeria_main_df.loc[np.isnan(nigeria_main_df.B6),'B6'] = 1
nigeria_main_df.loc[np.isnan(nigeria_main_df.B7),'B7'] = 0
nigeria_main_df.loc[np.isnan(nigeria_main_df.B9),'B9'] = 0
nigeria_main_df.loc[np.isnan(nigeria_main_df.B16),'B16'] = 0
nigeria_hh_df = nigeria_main_df[['HH_ID','B6','B7','B9','B16','B21']].rename(columns={'B6':'Households_share','B7':'Own_dwelling','B9':'Rooms_occupied','B16':'Bank_account','B21':'Mobile_money'})
nigeria_hh_df.head()

,HH_ID,Households_share,Own_dwelling,Rooms_occupied,Bank_account,Mobile_money
0,1.000506e+12,10.0,0.0,2.0,0.0,0.0
1,1.000506e+12,6.0,1.0,2.0,1.0,0.0
2,1.000506e+12,3.0,0.0,1.0,0.0,0.0
3,1.000506e+12,3.0,1.0,2.0,0.0,0.0
4,1.000506e+12,3.0,1.0,1.0,0.0,0.0


In [146]:
nigeria_electric_df = pd.read_stata('./Datasets/Nigeria_MTF_data/MTF_NG_HH_SEC_C.dta', convert_categoricals=False).drop(columns=['C_start']).drop(236)
nigeria_battery_df = pd.read_stata('./Datasets/Nigeria_MTF_data/MTF_NG_HH_SEC_C_BATTERY.dta', convert_categoricals=False)[['HH_ID','C114']].rename(columns={'C114':'battery_flag'})
nigeria_solar_df = pd.read_stata('./Datasets/Nigeria_MTF_data/MTF_NG_HH_SEC_C_SOLAR.dta', convert_categoricals=False)['HH_ID'].drop_duplicates().reset_index().drop(columns=['index'])
nigeria_solar_df['solar_flag'] = 1
nigeria_all_gen = pd.merge(pd.merge(nigeria_electric_df,nigeria_battery_df, how='left',on ='HH_ID'),nigeria_solar_df, how='left',on='HH_ID')
nigeria_all_gen.loc[pd.isna(nigeria_all_gen.battery_flag),'battery_flag'] = 0
nigeria_all_gen.loc[pd.isna(nigeria_all_gen.solar_flag),'solar_flag'] = 0
nigeria_all_gen['all_electric_flag'] = 1
nigeria_all_gen.loc[(nigeria_all_gen.solar_flag == 0) & (nigeria_all_gen.C2 == 0) & (nigeria_all_gen.battery_flag == 0),'all_electric_flag'] = 0
nigeria_electric_df_final = nigeria_all_gen[['HH_ID','C2','C_23','C_28a','C_28b','all_electric_flag','battery_flag','solar_flag']].rename(columns={'C2':'grid_connect','C_23':'average_energy_use_grid','C_28a':'Worst_hours_elect','C_28b':'typical_hours_elct'})


nigeria_all_mtf = pd.merge(nigeria_hh_df, nigeria_electric_df_final, how='left',on='HH_ID')

In [175]:
level2_boundary = geopandas.read_file("./Datasets/Shape files/nga_adm_osgof_20190417/nga_admbnda_adm2_osgof_20190417.shp")

In [213]:
level3_boundary = geopandas.read_file("./Datasets/Shape files/nga_adm_osgof_20190417/nga_admbnda_adm3_osgof_eha_20190417.shp")

In [222]:
level2_boundary.ADM1_EN.value_counts()

Borno      310
Adamawa    226
Yobe       178
Name: ADM1_EN, dtype: int64

In [216]:
level2_lookup=level2_boundary[['ADM1_EN','ADM2_EN']]

In [218]:
level2_lookup.ADM2_EN.drop_duplicates().sort_values()

21         Abadam
26     Askira/Uba
110          Bade
1            Bama
39           Bayo
          ...    
136        Toungo
10     Yola North
2      Yola South
93       Yunusari
92       Yusufari
Name: ADM2_EN, Length: 65, dtype: object

In [219]:
level2_lookup[level2_lookup.ADM2_EN == 'Kebbi']

,ADM1_EN,ADM2_EN


In [206]:
sum(level3_boundary.ADM3_EN.str.upper()== 'TURAKI HOUSE')

0

In [201]:
nigeria_ident

,HH_ID,lga,EA_code,state_id,ward,Urbanization,locality,EA_name,hhid,consent,int_start_time,language,Other_language,int_end_time
0,1.000506e+12,10005.0,1.000506e+08,6.0,1000503.0,1.0,SALAWA,MAIGARI SAIDU,10001.0,1.0,2017-11-20T06:47:00-05:00,2.0,,2017-11-20T07:57:06-05:00
1,1.000506e+12,10005.0,1.000506e+08,6.0,1000503.0,1.0,SALAWA,MAIGARI SAIDU,10003.0,1.0,2017-11-20T06:48:43-05:00,1.0,,2017-11-20T07:48:29-05:00
2,1.000506e+12,10005.0,1.000506e+08,6.0,1000503.0,1.0,SALAWA,MAIGARI SAIDU,10004.0,1.0,2017-11-20T08:03:34-05:00,2.0,,2017-11-20T09:08:46-05:00
3,1.000506e+12,10005.0,1.000506e+08,6.0,1000503.0,1.0,SALAWA,MAIGARI SAIDU,10005.0,1.0,2017-11-20T06:52:21-05:00,2.0,,2017-11-20T07:55:20-05:00
4,1.000506e+12,10005.0,1.000506e+08,6.0,1000503.0,1.0,SALAWA,MAIGARI SAIDU,10006.0,1.0,2017-11-20T06:49:41-05:00,2.0,,2017-11-20T07:39:52-05:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3664,1.200051e+13,120005.0,1.200051e+09,5.0,12000508.0,2.0,KANGIWA,NOMA AMADU,120010.0,1.0,2017-11-13T06:13:44-05:00,2.0,,2017-11-13T06:42:26-05:00
3665,1.200251e+13,120025.0,1.200251e+09,5.0,12002508.0,2.0,ARGUNGU,TURAKI HOUSE,120035.0,1.0,2017-11-14T05:35:14-05:00,1.0,,2017-11-14T06:20:07-05:00
3666,1.200751e+13,120075.0,1.200751e+09,5.0,12007508.0,1.0,MAIYAMA,G.R.A QUARTERS,120193.0,1.0,2017-11-21T05:14:45-05:00,1.0,,2017-11-21T06:24:05-05:00
3667,1.201450e+13,120145.0,1.201450e+09,5.0,12014504.0,2.0,YELWA YAURI,COBAS,120273.0,1.0,2017-12-04T05:48:17,2.0,,2017-12-04T08:39:40


In [162]:
nigeria_weights = pd.read_stata('./Datasets/Nigeria_MTF_data/ng_weights.dta', convert_categoricals=False)[['HH_ID','wt']]
nigeria_ident = pd.read_stata('./Datasets/Nigeria_MTF_data/MTF_NG_HH_Identification.dta', convert_categoricals=False)


In [164]:
mft_with_weights = pd.merge(pd.merge(nigeria_ident, nigeria_weights, how='inner',on='HH_ID'), nigeria_all_mtf, how = 'inner', on='HH_ID')
mft_with_weights

,HH_ID,lga,EA_code,state_id,ward,Urbanization,locality,EA_name,hhid,consent,...,Rooms_occupied,Bank_account,Mobile_money,grid_connect,average_energy_use_grid,Worst_hours_elect,typical_hours_elct,all_electric_flag,battery_flag,solar_flag
0,1.000506e+12,10005.0,1.000506e+08,6.0,1000503.0,1.0,SALAWA,MAIGARI SAIDU,10001.0,1.0,...,2.0,0.0,0.0,0.0,NaN,NaN,NaN,0,0.0,0.0
1,1.000506e+12,10005.0,1.000506e+08,6.0,1000503.0,1.0,SALAWA,MAIGARI SAIDU,10003.0,1.0,...,2.0,1.0,0.0,0.0,NaN,NaN,NaN,0,0.0,0.0
2,1.000506e+12,10005.0,1.000506e+08,6.0,1000503.0,1.0,SALAWA,MAIGARI SAIDU,10004.0,1.0,...,1.0,0.0,0.0,0.0,NaN,NaN,NaN,0,0.0,0.0
3,1.000506e+12,10005.0,1.000506e+08,6.0,1000503.0,1.0,SALAWA,MAIGARI SAIDU,10005.0,1.0,...,2.0,0.0,0.0,0.0,NaN,NaN,NaN,0,0.0,0.0
4,1.000506e+12,10005.0,1.000506e+08,6.0,1000503.0,1.0,SALAWA,MAIGARI SAIDU,10006.0,1.0,...,1.0,0.0,0.0,0.0,NaN,NaN,NaN,0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3663,1.200051e+13,120005.0,1.200051e+09,5.0,12000508.0,2.0,KANGIWA,NOMA AMADU,120010.0,1.0,...,5.0,0.0,0.0,0.0,NaN,NaN,NaN,0,0.0,0.0
3664,1.200251e+13,120025.0,1.200251e+09,5.0,12002508.0,2.0,ARGUNGU,TURAKI HOUSE,120035.0,1.0,...,4.0,1.0,0.0,1.0,NaN,NaN,22.0,1,0.0,0.0
3665,1.200751e+13,120075.0,1.200751e+09,5.0,12007508.0,1.0,MAIYAMA,G.R.A QUARTERS,120193.0,1.0,...,2.0,1.0,0.0,1.0,NaN,NaN,14.0,1,0.0,0.0
3666,1.201450e+13,120145.0,1.201450e+09,5.0,12014504.0,2.0,YELWA YAURI,COBAS,120273.0,1.0,...,2.0,1.0,0.0,1.0,NaN,10.0,15.0,1,0.0,0.0


In [142]:
print('NA values',sum(pd.isna(nigeria_all_gen.all_electric_flag)))
nigeria_all_gen.all_electric_flag.sort_values().value_counts()


NA values 0


0    1954
1    1714
Name: all_electric_flag, dtype: int64

In [46]:
raw_mtf_data['weightedhousehold_inhabitants'] = raw_mtf_data.pw_final*raw_mtf_data.household_size

raw_mtf_data.loc[np.isnan(raw_mtf_data.num_solar),'num_solar'] = 0
raw_mtf_data['solar_flag'] = 1
raw_mtf_data.loc[raw_mtf_data.num_solar==0,'solar_flag']=0
raw_mtf_data['weighted_solar'] = raw_mtf_data.pw_final*raw_mtf_data.num_solar
raw_mtf_data['weighted_solar_flag'] = raw_mtf_data.pw_final*raw_mtf_data.solar_flag
raw_mtf_data.loc[raw_mtf_data.generator_for_electricity==2,'generator_for_electricity'] = 0
raw_mtf_data['weighted_generator'] = raw_mtf_data.pw_final*raw_mtf_data.generator_for_electricity




,HH_ID,Households_share,Own_dwelling,Rooms_occupied,Bank_account,Mobile_money
0,1.000506e+12,10.0,0.0,2.0,0.0,0.0
1,1.000506e+12,6.0,1.0,2.0,1.0,0.0
2,1.000506e+12,3.0,0.0,1.0,0.0,0.0
3,1.000506e+12,3.0,1.0,2.0,0.0,0.0
4,1.000506e+12,3.0,1.0,1.0,0.0,0.0


In [91]:
Kenya_mft=pd.read_csv('./Datasets/MFT_KENYA_SUMMARY.csv')
Kenya_mft.columns

Index(['Unnamed: 0', 'county', 'num_households', 'total_inhabitants',
       'avg_household_size', 'avg_household_per_dwelling', 'pct_own_dwelling',
       'avg_rooms_occupied', 'percent_grid_connect', 'percent_any_elect',
       'percent_bank_account', 'percent_mobile_money',
       'total_percent_batteries', 'total_percent_solar',
       'total_percent_generator', 'grid_generator_pct', 'grid_battery_pct',
       'grid_solar_pct', 'avg_electricity_use_grid',
       'avg_typical_hours_elect_grid', 'avg_worst_hours_elect_grid',
       'not_grid_generator_pct', 'not_grid_battery_pct', 'not_grid_solar_pct',
       'avg_distance_to_connect'],
      dtype='object')

Index(['Unnamed: 0', 'county', 'num_households', 'total_inhabitants',
       'avg_household_size', 'avg_household_per_dwelling', 'pct_own_dwelling',
       'avg_rooms_occupied', 'percent_grid_connect', 'percent_bank_account',
       'percent_mobile_money', 'total_percent_batteries',
       'total_percent_solar', 'total_percent_generator', 'grid_generator_pct',
       'grid_battery_pct', 'grid_solar_pct', 'not_grid_generator_pct',
       'not_grid_battery_pct', 'not_grid_solar_pct',
       'avg_distance_to_connect'],
      dtype='object')